# SNP Grid

Access the SNP data stored in [BigQuery](https://cloud.google.com/bigquery/docs/).

## Import the libraries used in this tutorial

In [2]:
from google.cloud import bigquery

## Initialize a client

In [3]:
client = bigquery.Client(project = "jax-gedi-sandbox-nc-01")

To explicitly specify a project when constructing the client, set the `project` parameter:

## Query Examples

In [8]:
# query stain information
query = """
    SELECT 
      strainname, bq_sample_id
    FROM
      `jax-gedi-sandbox-nc-01.snp_v1.strain_info`
    WHERE bq_sample_id > 0
    LIMIT 10
"""
query_job = client.query(query)  
for row in query_job:
    print(f'{row["strainname"]} \t \t {row["bq_sample_id"]} ')

C3H/HeJ 	 	 6253071401682343689 
A/J 	 	 1570397219716225738 
BALB/cJ 	 	 2222972836691470842 
129S1/SvImJ 	 	 2622349345741473500 
FVB/NJ 	 	 7286594216810995812 
DBA/2J 	 	 8431637702450750551 
CAST/EiJ 	 	 2251371881872193158 
BTBR T<+> Itpr3<tf>/J 	 	 8109111734257395417 
C57BL/6J 	 	 6574517556882560158 


In [13]:
# query by chromosome, start position, strain name
query = """
    SELECT
      m.strainname,
      s.reference_name, s.start_position, s.reference_bases, s.names,
      call.sample_id, call.genotype, call.GQ, call.GN 
    FROM
      `jax-gedi-sandbox-nc-01.snp_v1.chr1__03_23000021_33000021` s, 
      UNNEST(call) as call,
      `jax-gedi-sandbox-nc-01.snp_v1.strain_info` m
    WHERE 
        s.reference_name = "1"
        AND s.start_position >= 23000295 AND s.start_position <= 23000411 
        AND m.strainname = 'C3H/HeJ'
        AND m.bq_sample_id = call.sample_id
    LIMIT 1000
"""
query_job = client.query(query)  
for row in query_job:
    print(f'{row["strainname"]} {row["reference_name"]} {row["start_position"]} \
        {row["reference_bases"]} {row["sample_id"]} {row["GN"]}')

C3H/HeJ 1 23000295         A 6253071401682343689 A
C3H/HeJ 1 23000312         A 6253071401682343689 A
C3H/HeJ 1 23000336         A 6253071401682343689 A
C3H/HeJ 1 23000404         A 6253071401682343689 A
C3H/HeJ 1 23000358         C 6253071401682343689 C
C3H/HeJ 1 23000371         C 6253071401682343689 C
C3H/HeJ 1 23000376         G 6253071401682343689 G
C3H/HeJ 1 23000315         T 6253071401682343689 T
C3H/HeJ 1 23000411         T 6253071401682343689 T


In [14]:
# to query across the table, first get the tables
query = """
    SELECT 
      chr, start_position, end_position, chr_table_name, strain_table_name
    FROM `jax-gedi-sandbox-nc-01.snp_v1.region_table_name`
    WHERE start_position > 13000019 and end_position < 53000023 
"""
query_job = client.query(query)  
for row in query_job:
    print(f'{row["chr"]} {row["start_position"]} {row["end_position"]} {row["chr_table_name"]} {row["strain_table_name"]}')

1 13000020 23000020 chr1__02_13000020_23000020 strain__02_13000020_23000020
1 23000021 33000021 chr1__03_23000021_33000021 strain__03_23000021_33000021
1 33000022 43000022 chr1__04_33000022_43000022 strain__04_33000022_43000022


In [15]:
query = "SELECT start_position FROM `jax-gedi-sandbox-nc-01.snp_v1.chr1__02_13000020_23000020` LIMIT 2"
query_job = client.query(query)  
for row in query_job:
    print(f'{row["start_position"]}')

13000190
13000048


In [16]:
query = "SELECT start_position FROM `jax-gedi-sandbox-nc-01.snp_v1.chr1__03_23000021_33000021` LIMIT 2"
query_job = client.query(query)  
for row in query_job:
    print(f'{row["start_position"]}')

23000295
23000312


In [17]:
query = "SELECT start_position FROM `jax-gedi-sandbox-nc-01.snp_v1.chr1__04_33000022_43000022` LIMIT 2"
query_job = client.query(query)  
for row in query_job:
    print(f'{row["start_position"]}')

33000481
33000203


In [20]:
query = """
    SELECT
      m.strainname,
      s.reference_name, s.start_position, s.reference_bases, s.names,
      call.sample_id, call.genotype, call.GQ, call.GN 
    FROM
      `jax-gedi-sandbox-nc-01.snp_v1.strain_info` m,
      (SELECT * FROM `jax-gedi-sandbox-nc-01.snp_v1.chr1__02_13000020_23000020` UNION ALL
      SELECT * FROM `jax-gedi-sandbox-nc-01.snp_v1.chr1__03_23000021_33000021` UNION ALL
      SELECT * FROM `jax-gedi-sandbox-nc-01.snp_v1.chr1__04_33000022_43000022`) s, 
      UNNEST(call) as call
    WHERE m.strainname = 'C3H/HeJ'
        AND m.bq_sample_id = call.sample_id
        AND start_position in (13000190, 23000295, 33000203)
    order by start_position
    LIMIT 1000
"""
query_job = client.query(query)  
for row in query_job:
    print(f'{row["strainname"]} {row["reference_name"]} {row["start_position"]} \
        {row["reference_bases"]} {row["sample_id"]} {row["GN"]}')

C3H/HeJ 1 13000190         A 6253071401682343689 A
C3H/HeJ 1 23000295         A 6253071401682343689 A
C3H/HeJ 1 33000203         A 6253071401682343689 A


In [27]:
# table
query = """
    SELECT
     * EXCEPT(is_generated, generation_expression, is_stored, is_updatable)
    FROM
     jax-gedi-sandbox-nc-01.snp_v1.INFORMATION_SCHEMA.COLUMNS
    WHERE
     table_name="strain_info"
"""
rows = client.query(query)  
for row in rows:
     print(f'{row["column_name"]} \t {row["data_type"]}')

strainname 	 STRING
vendor 	 STRING
stocknum 	 STRING
panel 	 STRING
mpd_strainid 	 INT64
straintype 	 STRING
n_proj 	 INT64
n_snp_datasets 	 INT64
mpd_shortname 	 STRING
mginum 	 STRING
url 	 STRING
bq_sample_id 	 INT64
